In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
import torchvision
from torchvision import datasets, models, transforms

In [6]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [7]:
from session import Session, TrainingSchedule
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import Validator
from callbacks import OneHotAccuracy
import Datasets.ImageClassifierData as ImageClassifierData

In [8]:
torch.cuda.is_available(), torch.cuda.get_device_name(0), torch.backends.cudnn.enabled

(True, 'GeForce GTX 770', True)

In [9]:
#DATA_PATH = Path('data/hymenoptera_data')
DATA_PATH = Path('data/dogscats')

In [10]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data = ImageClassifierData.from_paths(DATA_PATH, 16, data_transforms)

In [12]:
print(data['train'])

AttributeError: 'DataLoader' object has no attribute '__getitem__'

In [ ]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    figure, ax = plt.subplots()
    ax.imshow(inp)


'''
# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

imshow(out, title=[class_names[x] for x in classes])
'''

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
#model_ft.fc = nn.Linear(num_ftrs, 2)
model_ft.fc = nn.Sequential(
    nn.BatchNorm1d(num_features=num_ftrs),
    nn.Dropout(p=.5),
    nn.Linear(num_ftrs, 2),
    nn.LogSoftmax(dim=1)
)

In [ ]:
# model_ft

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=1e-3)

In [ ]:
sess = Session(model_ft, criterion, optimizer)
sess.freeze()

In [ ]:
lr_scheduler = CosAnneal(len(data['train']), T_mult=2)
validator = Validator(data['valid'], OneHotAccuracy())

In [ ]:
schedule = TrainingSchedule(data['train'], callbacks=[lr_scheduler, validator])

In [ ]:
lr_find(sess, data['train'], start_lr=1e-5)

In [ ]:
sess.set_lr(1e-2)

In [ ]:
sess.train(schedule, 3)

In [ ]:
sess.unfreeze()

In [ ]:
lr_find(sess, data['train'], start_lr=1e-5)

In [ ]:
sess.set_lr(5e-3)

In [ ]:
sess.train(schedule, 3)

In [ ]:
lr_scheduler.plot()